In [ ]:
# Notebook 1: Data Ingestion and Chunking
# Ce notebook montre comment extraire et découper des documents
import docx
# Cellule 1: Imports
import sys
sys.path.append('../src')

from modules.ingestion import DocumentIngestion
from modules.chunking import TextChunker
from modules.config import config
from pathlib import Path
import pandas as pd

print("✅ Modules importés avec succès")

# Cellule 2: Initialisation
ingestion = DocumentIngestion()
chunker = TextChunker(chunk_size=500, chunk_overlap=50)

print(f"Chunk size: {chunker.chunk_size}")
print(f"Chunk overlap: {chunker.chunk_overlap}")

# Cellule 3: Exemple avec un fichier texte simple
sample_text = """
L'intelligence artificielle (IA) est un ensemble de théories et de techniques 
mises en œuvre en vue de réaliser des machines capables de simuler l'intelligence humaine.

Le machine learning est une branche de l'IA qui permet aux ordinateurs d'apprendre 
à partir de données sans être explicitement programmés.

Le deep learning est une sous-catégorie du machine learning qui utilise des 
réseaux de neurones artificiels avec plusieurs couches cachées.
"""

# Tester le chunking
chunks = chunker.split_into_chunks(sample_text)
print(f"\n📊 Nombre de chunks créés: {len(chunks)}")
print("\n📄 Premier chunk:")
print(chunks[0])

# Cellule 4: Créer des chunks avec métadonnées
chunks_with_meta = chunker.create_chunks_with_metadata(
    sample_text, 
    "exemple.txt"
)

# Afficher sous forme de DataFrame
df_chunks = pd.DataFrame(chunks_with_meta)
print("\n📊 Chunks avec métadonnées:")
display(df_chunks[['chunk_id', 'chunk_index', 'num_words', 'num_characters']])

# Cellule 5: Traiter un vrai document
# Assurez-vous d'avoir un fichier PDF dans data/documents/
doc_path = config.DOCUMENTS_DIR / "votre_document.pdf"  # Remplacer par un vrai fichier

if doc_path.exists():
    print(f"📄 Traitement de: {doc_path.name}")
    
    # Extraction
    doc_info = ingestion.process_document(doc_path)
    print(f"✅ Texte extrait: {doc_info['num_characters']} caractères")
    
    # Chunking
    chunks = chunker.create_chunks_with_metadata(
        doc_info['content'],
        doc_info['filename']
    )
    print(f"✅ Chunks créés: {len(chunks)}")
    
    # Visualiser
    df = pd.DataFrame(chunks)
    display(df.head())
else:
    print("⚠️ Veuillez placer un document PDF dans data/documents/")

# Cellule 6: Statistiques sur les chunks
if 'chunks' in locals():
    stats = {
        'total_chunks': len(chunks),
        'avg_words': sum(c['num_words'] for c in chunks) / len(chunks),
        'avg_chars': sum(c['num_characters'] for c in chunks) / len(chunks),
        'min_words': min(c['num_words'] for c in chunks),
        'max_words': max(c['num_words'] for c in chunks)
    }
    
    print("\n📊 Statistiques des chunks:")
    for key, value in stats.items():
        print(f"{key}: {value:.2f}")

# Cellule 7: Sauvegarder les chunks (optionnel)
import json

if 'chunks' in locals():
    output_path = config.CHUNKS_DIR / f"{doc_info['filename']}_chunks.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    print(f"✅ Chunks sauvegardés: {output_path}")